In [8]:
import numpy as np
import json
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, GRU, Dropout, Dense


def predict_next_day_values(data_path, get_value: str):
    # Load data from JSON file
    with open(data_path) as f:
        data = json.load(f)

    # Extract features and target
    X = np.column_stack((data["o"], data["h"], data["l"], data["c"], data["v"]))
    y = np.array(data[get_value])

    # Normalize data
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

    # Reshape data for LSTM input (samples, timesteps, features)
    X_scaled = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)

    # Build the model
    model = Sequential()
    model.add(
        Bidirectional(
            GRU(units=50, return_sequences=True, input_shape=(X_scaled.shape[1], 1))
        )
    )
    model.add(Dropout(0.2))
    model.add(Bidirectional(GRU(units=50)))
    model.add(Dropout(0.2))
    model.add(Dense(32, kernel_initializer="uniform", activation="relu"))
    model.add(Dense(1, kernel_initializer="uniform", activation="linear"))
    model.compile(optimizer="adam", loss="mean_squared_error")

    # Train the model using all data
    model.fit(X_scaled, y_scaled, epochs=200, batch_size=32, verbose=1)

    # Predict tomorrow's value
    last_data_point = X_scaled[-1].reshape(1, X_scaled.shape[1], 1)
    predicted_value_scaled = model.predict(last_data_point)
    predicted_value = scaler_y.inverse_transform(predicted_value_scaled)

    return predicted_value


# Define data file path
data_path = "hsg.json"

# Define values to predict
values_to_predict = ["o"]

# Predict for each value
for value in values_to_predict:
    predicted_value = predict_next_day_values(data_path, value)
    print(f"Predicted {value} for tomorrow:", predicted_value)

Epoch 1/200
120/120 [==============================] - 10s 28ms/step - loss: 0.0112
Epoch 2/200
120/120 [==============================] - 3s 26ms/step - loss: 3.1458e-04
Epoch 3/200
120/120 [==============================] - 3s 25ms/step - loss: 9.9872e-05
Epoch 4/200
120/120 [==============================] - 3s 25ms/step - loss: 3.9804e-05
Epoch 5/200
120/120 [==============================] - 3s 24ms/step - loss: 2.5235e-05
Epoch 6/200
120/120 [==============================] - 3s 25ms/step - loss: 2.2039e-05
Epoch 7/200
120/120 [==============================] - 3s 26ms/step - loss: 2.0575e-05
Epoch 8/200
120/120 [==============================] - 3s 27ms/step - loss: 1.8652e-05
Epoch 9/200
120/120 [==============================] - 3s 24ms/step - loss: 1.8727e-05
Epoch 10/200
120/120 [==============================] - 3s 25ms/step - loss: 1.9660e-05
Epoch 11/200
120/120 [==============================] - 3s 25ms/step - loss: 1.6338e-05
Epoch 12/200
120/120 [======================